In [1]:
import cvxpy as cp
import numpy as np

from basis_generator import basis_gs, sel_seq

# Guess Your Neighboors Input parameters

In [2]:
dim = 2
num_obs = 2
len_seq = 3
out_max = 1
basis_size= 15
seq_method="sel_sequences"

GYN_events = [((0, 0, 0), (0, 0, 0)),
            ((1, 1, 0), (0, 1, 1)),
            ((0, 1, 1), (1, 0, 1)),
            ((1, 0, 1), (1, 1, 0)),]

# Optimization with Unrestricted Dimension

In [10]:
def comp_seq(ind1,ind2,seq):
    
    S = [list(seq[ind2][0][::-1]+seq[ind1][0]),list(seq[ind2][1][::-1]+seq[ind1][1])]

    ind = 1
    while ind < len(S[0]):
        if S[0][ind-1] == S[0][ind] and S[1][ind-1] == S[1][ind]:
            del S[0][ind]
            del S[1][ind]
        elif S[0][ind-1] != S[0][ind] and S[1][ind-1] == S[1][ind]:
            return []
        else:
            ind+=1
    return S

In [11]:
seq = sel_seq(num_obs = 3,
                len_seq = 2,
                num_out = 2,
                remove_last_out = False)
n = len(seq)

In [20]:
M = cp.Variable((n,n),symmetric=True)

constraints = [M >> 0]
# constraints += [
#     cp.trace(X) == num_obs**len_seq
# ]

In [16]:
for ind11 in range(len(seq)):
    for ind12 in range(ind11,len(seq)):
        for ind21 in range(len(seq)):
            for ind22 in range(ind21,len(seq)):
                #print([ind11,ind12],[ind21,ind22])
                S1 = comp_seq(ind11,ind12,seq)
                S2 = comp_seq(ind21,ind22,seq)
                if S1 == S2:
                    constraints += [M[ind11,ind12] == M[ind21,ind22]]
                    constraints += [M[ind12,ind11] == M[ind22,ind21]]
                elif len(S1)==0:
                    constraints += [M[ind11,ind12] == 0]
                    constraints += [M[ind12,ind11] == 0]
                elif len(S2)==0:
                    constraints += [M[ind21,ind22] == 0]
                    constraints += [M[ind22,ind21] == 0]

In [18]:
C = np.zeros((n,n))
for i, seq_row in enumerate(sequences):
    if seq_row in GYN_events:
        C[i,i] = 1
        
prob = cp.Problem(cp.Maximize(cp.trace(C@M)),
                  constraints)

In [21]:
prob.solve(solver=cp.MOSEK, verbose=True)

print(prob.status)
print("The optimal value is", prob.value)



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 930836          
  Cones                  : 0               
  Scalar variables       : 1296            
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 930836          
  Cones                  : 0               
  Scalar variables       : 1296            
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer  - threads                : 12              
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 1296
Optimizer  - Cones                  : 1
Optimizer  - Scalar variable

# Optimization with the First Level of the Hierarchy

In [3]:
X_basis = basis_gs(dim = 2,
            num_obs = 3,
            len_seq = 2,
            num_out = 2,
            prec = 1e-15,
            stop = 10000)

Nul matrix found
Number of LI moment matrices:  52


In [4]:
alpha = cp.Variable((len(X_basis), 1))
X = cp.Variable(X_basis[0].shape, symmetric=True)

constraints = [X >> 0]
constraints += [
    X == sum([alpha[i]*X_basis[i] for i in range(len(X_basis))])
]
constraints += [
    cp.trace(X) == num_obs**len_seq
]

In [5]:
sequences = sel_seq(num_obs = 3,
                len_seq = 2,
                num_out = 2,
                remove_last_out = False)

C = np.zeros(X_basis[0].shape)
for i, seq_row in enumerate(sequences):
    if seq_row in GYN_events:
        C[i,i] = 1
        
prob = cp.Problem(cp.Maximize(cp.trace(C@X)),
                  constraints)

In [6]:
prob.solve(solver=cp.MOSEK, verbose=False)

print(prob.status)
print("The optimal value is", prob.value)

optimal
The optimal value is -0.0
